In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow import keras

In [3]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

## remember, "connect-windows"

In [4]:
batch_size = 32 
img_height = 224
img_width = 224 

In [5]:
validation_d = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 50947 files belonging to 7 classes.


In [6]:
training_d = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 399839 files belonging to 7 classes.


In [7]:
def preprocess(image, label):
    final_image = keras.applications.xception.preprocess_input(image)
    return final_image, label

In [8]:
training_ds = training_d.map(preprocess).batch(batch_size).prefetch(1)
validation_ds = validation_d.map(preprocess).batch(batch_size).prefetch(1)

In [9]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.
Found 50947 files belonging to 7 classes.


In [10]:
model_1 = keras.models.load_model('0825_model.h5')

In [11]:
model_2 = keras.models.load_model('0901_mobilenet.h5')

In [12]:
model_3 = keras.models.load_model('0906_EffiB4.h5')

In [15]:
model_2.evaluate(validation_s)

1593/1593 [==============================] - 45s 28ms/step - loss: 0.6359 - accuracy: 0.7731


[0.6359342932701111, 0.7731367945671082]

In [16]:
model_3.evaluate(validation_s)

1593/1593 [==============================] - 239s 148ms/step - loss: 0.5155 - accuracy: 0.8228


[0.515472948551178, 0.8227570056915283]

In [24]:
Error = {}
j=0
for image, label in validation_s:
    A = image
    B = label
    Pr = np.round(model_3.predict(A), 2)
    Tr = np.array(B)
    for i in range(len(B)):
        if np.argmax(Pr[i]) != Tr[i]:
            Error[j] = str(Pr[i]) + str(Tr[i])
            j = j+1

1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 2s 2s/step


In [27]:
C = model_3.predict(validation_s)

1593/1593 [==============================] - 230s 145ms/step


In [28]:
D = np.round(C, 2)

In [29]:
E = pd.DataFrame(D)

In [30]:
F = pd.DataFrame.from_dict(Error, orient = 'index')

In [31]:
E.to_csv('0908_B4_prob.csv')
F.to_csv('0908_B4_err.csv')